In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\abhis\\OneDrive\\Desktop\\Oil_Retail\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\abhis\\OneDrive\\Desktop\\Oil_Retail'

In [5]:
import pandas as pd 

In [6]:
data = pd.read_csv("artifacts\data_ingestion\oil_retail_history.csv")

In [7]:
data.head()

,date,price,cost,comp1_price,comp2_price,comp3_price,volume
0,2023-01-01,97.66,86.46,98.39,99.10,97.37,13513
1,2023-01-02,96.46,87.25,95.37,96.25,97.38,13605
2,2023-01-03,94.67,86.40,94.41,94.69,94.50,16540
3,2023-01-04,97.32,87.51,99.12,96.90,97.54,14927
4,2023-01-05,94.08,86.38,93.31,94.61,93.85,13332


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         730 non-null    object 
 1   price        730 non-null    float64
 2   cost         730 non-null    float64
 3   comp1_price  730 non-null    float64
 4   comp2_price  730 non-null    float64
 5   comp3_price  730 non-null    float64
 6   volume       730 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 40.0+ KB


In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [19]:
from oil_retail.constants import *
from oil_retail.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [21]:
import os 
from oil_retail import logger

In [22]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [23]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[ 2025-11-25 15:34:18,797 ] oil_retail - INFO - common.py:31 - yaml file: config\config.yaml loaded successfully
[ 2025-11-25 15:34:18,802 ] oil_retail - INFO - common.py:31 - yaml file: params.yaml loaded successfully
[ 2025-11-25 15:34:18,809 ] oil_retail - INFO - common.py:31 - yaml file: schema.yaml loaded successfully
[ 2025-11-25 15:34:18,812 ] oil_retail - INFO - common.py:51 - created directory at: artifacts
[ 2025-11-25 15:34:18,815 ] oil_retail - INFO - common.py:51 - created directory at: artifacts/data_validation
